In [ ]:
from dotenv import load_dotenv
from src.create_kb import create_kb_from_file
from langchain.chat_models import init_chat_model
from langgraph.graph import START, StateGraph
from typing_extensions import TypedDict

llm = init_chat_model("gpt-4o-mini", model_provider="openai")

In [3]:
load_dotenv()
# create knowledge base from transcript
file_path = "transcript.txt"
kb_id = "transcript"
kb = create_kb_from_file(kb_id, file_path)

Creating KB with id transcript...


In [4]:
# Define state for application
class State(TypedDict):
    question: str
    context: list[dict]
    answer: str

In [14]:
# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}

In [15]:
# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [16]:
response = graph.invoke({"question": "What is Task Decomposition?"})
print(response["answer"])

Task decomposition is the process of breaking down complex tasks into smaller, manageable steps. This can be achieved through techniques such as simple prompting, task-specific instructions, or human inputs. Additionally, methods like Chain of Thought (CoT) and Tree of Thoughts facilitate this breakdown by encouraging step-by-step reasoning and exploring multiple possibilities.
